In [1]:
import sklearn
print(sklearn.__version__)

0.20.0


In [2]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from time import time

import numpy as np
import pandas as pd
#from functions.models import 
from Models.functions.datasets import getDatasets
from Models.functions.metrics import evaluator
from Models.functions.plot import ROC, plot_confusion_matrix
from Models.functions.preprocessing import clean

In [3]:
%matplotlib inline

In [35]:
dataset_name = 'brmoral'
task = 'polit'

def getBestParams(task, dataset_name):
    baseline = 'baseline1'
    dataset_name = dataset_name.strip().lower()
    task = task.strip().lower()
    
    # load excel params
    baseline1 = pd.read_excel('../Reports/Reports.xlsx', baseline)
    baseline1['Task'] = baseline1['Task'].str.lower()
    baseline1['Name'] = baseline1['Name'].str.lower()
    
    best_params = baseline1[(baseline1['Name'] == dataset_name) & (baseline1['Task'] == task)]
    
    max_features = best_params['max features'].values[0]
    
    model_params = {
                    'vect__max_features': max_features if max_features != 'None' and not pd.isna(max_features) else None,
                    'vect__max_df': best_params['max df'].values[0] if not pd.isna(best_params['max df'].values[0]) else 1,
                    'clf__C': best_params['C'].values[0] if not pd.isna(best_params['C'].values[0]) else 1000.0, 
                    'clf__penalty': best_params['P'].values[0] if not pd.isna(best_params['P'].values[0]) else 'l2'
                    }
    
    return model_params

getBestParams(task, dataset_name)

{'clf__C': 10000,
 'clf__penalty': 'l1',
 'vect__max_df': 0.8,
 'vect__max_features': 3000}

In [36]:
def labelEncoder(y):
    le = LabelEncoder()
    le.fit(y)

    return (le.transform(y), len(le.classes_), list(le.classes_))

### Perform classification for each problem / task

In [37]:
def model(X, y, n_classes, classes_name, params):
    
    # pipeline.set_params(**params)    
    vect = TfidfVectorizer(max_features=params.get('vect__max_features'), max_df=params.get('vect__max_df'))
    
    K = StratifiedKFold(n_splits=10)
    
    t0 = time()
    
    predicted_y = []
    expected_y = []    
    score_y = []
    
    for train_index, test_index in K.split(X, y):
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
        X_train = vect.fit_transform(X_train)
        X_test = vect.transform(X_test)
        
        clf = LogisticRegression(C=params.get('clf__C'), penalty=params.get('clf__penalty'), solver='liblinear')
        
        clf.fit(X_train, y_train)
        
        predicted_y.extend(clf.predict(X_test))
        expected_y.extend(y_test)
        score_y.extend(clf.predict_proba(X_test))

        ### get train score

    # print("done in %0.2fs and %0.1fmin" % ((time() - t0), ((time() - t0) / 60) ))
    # print()
    
    report = pd.DataFrame(classification_report(expected_y, predicted_y, digits=5, target_names=classes_name, output_dict=True))
    report = report.transpose()
    
    return (
        report, 
        np.asarray(expected_y),
        np.asarray(predicted_y),
        np.asarray(score_y)
        )

In [43]:
def run(task, dataset_name, output = None):    
    datasets = getDatasets(task,'df', dataset_name)
    for i in datasets.iterrows():

        name = i[1]['dataset_name']
        label = task
        ds_path = i[1]['path']

        # load training and test dataframes
        training_path = ds_path + '/' + i[1]['training']        

        df_training = pd.read_csv(training_path)#, usecols=cols)        
        
        df_training['text'] = df_training['text'].apply(clean)
        
        X_train = df_training['text'].values
        y_train, n_classes, classes_name = labelEncoder(df_training[label].values)

        # del(df_training)

        # print("Dataset: {0} and task: {1}".format(name, label))

        # print("n_classes: {0}".format(n_classes))

        params = getBestParams(task, dataset_name)
        if dataset_name == 'brmoral' and task == 'relig':        
            params['clf__C'] = 1000
        #params['clf__penalty'] = 'l1'
        #params['clf__maxiter'] = '500'
        print("params: ", params)

        report, expected_y, predicted_y, score_y = model(X_train, y_train, n_classes, classes_name, params)

        # get ROC
        roc_c = ROC(expected_y, score_y, n_classes, task, dataset_name, classes_name)
        report['roc'] = list(roc_c.values()) + [roc_c['macro']] * 2

        # compute accuracy
        accuracy = accuracy_score(expected_y, predicted_y)
        report['accuracy'] = [accuracy] * (n_classes + 3)

        # compute confusion matrix
        c_matrix = confusion_matrix(expected_y, predicted_y)
        plot_confusion_matrix(c_matrix, classes_name, task, dataset_name, True)
        cm = pd.DataFrame(c_matrix, columns=classes_name, index=classes_name)

        directory = './Reports/' + task + '/' + dataset_name + '/'
        report.to_csv(directory + 'report.csv')
        cm.to_csv(directory + 'confusion_matrix.csv')    

        print(task, dataset_name, report)
        print(cm)

        # output.put('results for {0} and {1}'.format(task, dataset_name))
        # output.put(report.to_dict())

        pass

In [45]:
import multiprocessing as mp
import random
import string

random.seed(123)

# Define an output queue
output = mp.Queue()

task_list = ['relig','polit','education','professional','region','TI','gender','age']
dataset_list = ['brmoral','b5post','esic','brblogset','enblogs','pan13_en','pan13_es']

args = []
for task in task_list:
    for ds in dataset_list:
        d = getDatasets(task,'df', ds)
        if len(d.values) > 0:
            args.append([task, ds])

In [8]:
# Setup a list of processes that we want to run
processes = [mp.Process(target=run, args=(x[0], x[1], output)) for x in args]

# Run processes
for p in processes:
    p.start()

# Exit the completed processes
for p in processes:
    p.join()

# Get process results from the output queue
results = [output.get() for p in processes]

print(results)

/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/rafael/.lo

Normalized confusion matrix
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Normalized confusion matrix
[[0.53846154 0.37362637 0.08791209]
 [0.30434783 0.44565217 0.25      ]
 [0.24096386 0.31325301 0.44578313]]
relig brmoral               f1-score  precision    recall  support  roc  accuracy
r12           0.705570   0.545082  1.000000    133.0  0.5  0.545082
r3            0.000000   0.000000  0.000000     90.0  0.5  0.545082
r45           0.000000   0.000000  0.000000     21.0  0.5  0.545082
micro avg     0.545082   0.545082  0.545082    244.0  0.5  0.545082
macro avg     0.235190   0.181694  0.333333    244.0  0.5  0.545082
weighted avg  0.384594   0.297114  0.545082    244.0  0.5  0.545082


/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


age brmoral               f1-score  precision    recall  support       roc  accuracy
a17-22        0.521277   0.505155  0.538462     91.0  0.706499  0.477444
a25-29        0.424870   0.405941  0.445652     92.0  0.548913  0.477444
a31-46        0.490066   0.544118  0.445783     83.0  0.710975  0.477444
micro avg     0.477444   0.477444  0.477444    266.0  0.658402  0.477444
macro avg     0.478738   0.485071  0.476632    266.0  0.658402  0.477444
weighted avg  0.478195   0.482998  0.477444    266.0  0.658402  0.477444Normalized confusion matrix

[[0.49193548 0.27419355 0.23387097]
 [0.32692308 0.40384615 0.26923077]
 [0.39175258 0.34020619 0.26804124]]
Normalized confusion matrix
[[0.47058824 0.52941176]
 [0.1407767  0.8592233 ]]Normalized confusion matrix

[[0.34579439 0.65420561]
 [0.15137615 0.84862385]]
education brmoral                                      f1-score  precision    recall  support  \
Básico + Superior incompleto         0.474708   0.458647  0.491935    124.0   
Pós-gr

/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
Process Process-10:
Process Process-12:
Process Process-16:
Process Process-15:
Process Process-21:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-13:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Process Process-22:
Process Process-17:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Process-14:
  File "/usr/lib/python3.6/mult

  File "/home/rafael/drive/Models/functions/preprocessing.py", line 7, in clean
    def clean(doc, lang = 'portuguese'):
  File "/home/rafael/.local/lib/python3.6/site-packages/nltk/tokenize/simple.py", line 138, in line_tokenize
    return LineTokenizer(blanklines).tokenize(text)
KeyboardInterrupt
  File "/home/rafael/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py", line 265, in <lambda>
    return lambda doc: token_pattern.findall(doc)
  File "/home/rafael/drive/Models/functions/preprocessing.py", line 17, in clean
    stop_words = set(stopwords.words(lang))
  File "/home/rafael/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py", line 265, in <lambda>
    return lambda doc: token_pattern.findall(doc)
  File "/home/rafael/drive/Models/functions/preprocessing.py", line 31, in clean
    tokens = doc.split()
KeyboardInterrupt
  File "/home/rafael/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py", line 308, in <lambda>
    to

KeyboardInterrupt: 

Process Process-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-b4ff2cee5612>", line 28, in run
    report, expected_y, predicted_y, score_y = model(X_train, y_train, n_classes, classes_name, params)
  File "<ipython-input-5-4749a44d79b4>", line 24, in model
    clf.fit(X_train, y_train)
  File "/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py", line 1301, in fit
    sample_weight=sample_weight)
  File "/home/rafael/.local/lib/python3.6/site-packages/sklearn/svm/base.py", line 914, in _fit_liblinear
    epsilon, sample_weight)
KeyboardInterrupt
/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to